In [26]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [27]:
data = pd.read_csv('train.csv')

lemmatizer = WordNetLemmatizer()
# defining own Stopwords with the help of stopwords in nltk
stop = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z',    'yourself', 'more', 'if', 'on', 'don', "don't", 'once', 'this', 'being', 'as', 'there', 'should', 'its', 'been', 'didn', "you're", 'me', 'but', 'than', 'just', 'an', 'when', 'after', 'now', 'your', "needn't", 'aren', 'that', 'during', 've', 'yourselves', "you'll", 'who', 'whom', "hadn't", "weren't", "shouldn't", 'at', 'the', 'can', 'herself', 'from', 'those', "she's", 'hers', 'so', 'for', 'll', 'do', "didn't", 't', 's', 'to', 'wouldn', "hasn't", 'before', "shan't", 'too', "mightn't", 'above', 'most', 'him', 'theirs', 'has', 'she', 'i', 'here', 'be', 'because', 'd', 'y', "couldn't", 'doesn', "wouldn't", 'ma', 'all', 'doing', 'himself', 'are', 'o', "doesn't", 'what', 'my', 'up', "you've", 'nor', 'couldn', 'ours', 'his', 'themselves', 'which', 'in', 'having', "isn't", 'while', 'shan', 'below', 'ain', "won't", 'these', 'needn', "it's", 'why', 'were', 'mightn', 'won', 'itself', 'mustn', 'was', 'against', 'of', 'then', 'both', "wasn't", 'by', 'or', 'myself', 'a', 'only', 'we', 'down', 'no', 'between', 'some', 'hadn', 'where', 'until', 'other', 'did', 'they', 'have', "haven't", 'further', 'you', 'had', 'yours', 'through', 'same', "should've", 'he', 'off', 'will', 'few', 'ourselves', 'how', "aren't", 'wasn', 'our', 'isn', 'them', 'into', "that'll", "mustn't", 'such', 'their', 'hasn', 'm', 'weren', 'about', 'under', 'it', 'does', 'not', 'haven', "you'd", 'over', 'out', 'any', 'and', 'each', 'very', 'her', 'with', 'own', 're', 'shouldn', 'am', 'again', 'is']

def preprocess(user_text):
    ind = ' '.join(re.findall(r'\w+|\d+', user_text))
    ind = re.sub(r"http:[^\s]*", "", ind)
    ind = ind.lower()
    ind = word_tokenize(ind)
    ind = [lemmatizer.lemmatize(word) for word in ind if word not in stop]
    return " ".join(ind)

data['text'] = data['text'].apply(preprocess)
tfidf = TfidfVectorizer()
X = tfidf.fit_transform(data['text'])
Y = data['target']

In [31]:
## Model Fitting:

### Train and Validation Set:

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier




X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state=0)

svc = SVC(kernel='rbf')
rndReg = RandomForestClassifier(n_estimators=10,random_state=1)
dTree = DecisionTreeClassifier(max_depth=5,min_samples_split=4,min_samples_leaf=4)
lg = LogisticRegression(random_state=1)
model_GB =  GradientBoostingClassifier(random_state=1)
model_list,rmse,acc,R2Score,AvgCVScore,AvgCVRMSE = [],[],[],[],[],[]
models =[lg,dTree,rndReg,svc,model_GB]
for model in models:
    print(model)
    model_list.append(model.fit(X_train,y_train))
    model1 = model.fit(X_train,y_train)
    y_predict = model.predict(X_test)
    rmse.append(np.sqrt(mean_squared_error(y_test,y_predict)))
    acc.append(model.score(X_test,y_test))
    R2Score.append(r2_score(y_predict,y_test))
    #10-fold Cross Validation
    scores = cross_val_score(model1,X_train, y_train, scoring='r2', cv = 10)
    AvgCVScore.append(scores.mean())
    AvgCVRMSE.append(np.sqrt(-cross_val_score(model1,X_train, y_train,
                                      scoring='neg_mean_squared_error', cv = 10)).mean())

model_score = pd.DataFrame({'Model':['Logistic Regression','Decision Tree','Random Forest','SVM','Gradient Boosting'],
                            'RMSE':rmse,'Accuracy':acc,'Avg CV RMSE':AvgCVRMSE,
                            'Avg CV Score': AvgCVScore,'R2 Score':R2Score})
print(model_score)

LogisticRegression(random_state=1)
DecisionTreeClassifier(max_depth=5, min_samples_leaf=4, min_samples_split=4)
RandomForestClassifier(n_estimators=10, random_state=1)


KeyboardInterrupt: 